In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
# import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
tracks = pd.read_csv("data/tracks.csv",index_col=0)
tracks.dropna(inplace=True)
tracks.reset_index(inplace=True)
del tracks["index"]
tracks



In [ ]:
X = tracks['lyrics']
y = tracks['explicit']
# Getting training and testing X and y

# Convert text to numerical features
vectorizer = CountVectorizer()
Xvec = vectorizer.fit_transform(X)

# Split into training and testing
trainX, testX, trainY, testY = train_test_split(Xvec,y)

# Train the classifier
classifier = MultinomialNB()
classifier.fit(trainX,trainY)

# Predict the test data
predictY = classifier.predict(testX)

# Get metrics for classifier (precision, recall, fscore, support)
metrics = {}
p,r,f,s = precision_recall_fscore_support(testY,predictY)

metrics["precision"] = p
metrics["recall"] = r
metrics["f-score"] = f
metrics["support"] = s

for metric in metrics.keys():
    print(f"{metric}: {metrics[metric]}")